In [42]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

In [43]:
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()

In [44]:
df["spi_margin"] = np.abs(df["spi1"] - df["spi2"])

In [45]:
spi = []

In [46]:
msm = np.mean(df["spi_margin"])
mssd = np.std(df["spi_margin"])
msm15sd = msm + (mssd * 1.5)
msm2sd = msm + (mssd * 2)

In [47]:
for x in df["spi_margin"]:
    if x < msm:
        spi.append("CLOSE SPI MATCHUP")
    elif (x >= msm) & (x < msm15sd):
        spi.append("SLIGHT SPI FAVORITE")
    elif (x >= msm15sd) & (x < msm2sd):
        spi.append("CLEAR SPI FAVORITE")
    else:
        spi.append("SPI BLOWOUT")

In [48]:
df["spi_bucket"] = spi

In [49]:
X = df[['spi1', 'spi2', 'spi_margin']]
y = df[['spi_bucket']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [50]:
model = XGBClassifier(n_estimators=400, random_state=6, label_encoder=False).fit(X_train, y_train)

c:\users\henry\miniconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\henry\miniconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[01:13:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "label_encoder" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:13:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [51]:
predictions = model.predict(X_test)

In [52]:
accuracy = accuracy_score(y_test, predictions)

In [54]:
print("Test Accuracy: {}%".format(accuracy * 100))

Test Accuracy: 100.0%
